In [2]:
!pip install netmiko
!pip install "paramiko[all]"
!pip install pywin32

Paramiko é muito útil para fazer as conexões de modo simples, mas tem muitas dependências que podem levar ao desuso ou erros genericos aleatórios.
Netmiko essa deve ser a bilbioteca central para fazer as operações pela simplicidade, para fazer a conexão é passado um dicionário.

# NetMiko Suport:

Regularly tested

Arista vEOS

Cisco ASA

Cisco IOS

Cisco IOS-XE

Cisco IOS-XR

Cisco NX-OS

Cisco SG300

HP ProCurve

Juniper Junos

Linux

Limited testing

6Wind

Adtran OS

Alcatel AOS6/AOS8

Apresia Systems AEOS

Broadcom ICOS

Calix B6

Centec Networks

Cisco AireOS (Wireless LAN Controllers)

CloudGenix ION

Dell OS9 (Force10)

Dell OS10

Dell PowerConnect

Ericsson IPOS

Extreme ERS (Avaya)

Extreme VSP (Avaya)

Extreme VDX (Brocade)

Extreme MLX/NetIron (Brocade/Foundry)

HPE Comware7

Huawei

Huawei OLT

Huawei SmartAX

IP Infusion OcNOS

Juniper ScreenOS

Mellanox

MikroTik RouterOS

MikroTik SwitchOS

NetApp cDOT

Netgear ProSafe

Nokia/Alcatel SR OS

OneAccess

Palo Alto PAN-OS

Pluribus

Ruckus ICX/FastIron

Ruijie Networks

Supermicro SMIS

TPLink JetStream

Ubiquiti EdgeSwitch

Vyatta VyOS

Yamaha

ZTE ZXROS

Experimental

A10

Accedian

Allied Telesis AlliedWare Plus

Aruba

Brocade Fabric OS

C-DOT CROS

Ciena SAOS

Citrix Netscaler

Cisco Telepresence

Cisco Viptela

Check Point GAiA

Coriant

Dell OS6

Dell EMC Isilon

Eltex

Enterasys

Endace

Extreme EXOS

Extreme Wing

Extreme SLX (Brocade)

F5 TMSH

F5 Linux

Fortinet

MRV Communications OptiSwitch

MRV LX

Nokia/Alcatel SR-OS

QuantaMesh

Rad ETX

Raisecom ROAP

Sophos SFOS

Ubiquiti Unifi Switch

Versa Networks FlexVNF

Watchguard Firebox

6WIND TurboRouter

https://github.com/ktbyers/netmiko

In [3]:
import paramiko
from netmiko import ConnectHandler
from paramiko.ssh_exception import AuthenticationException

Usando paramiko:
```
def conectaSwitch():
	# Definições de login e senha para os switches
	login = 'usuario'
	pwd = 'senha1'
	pwd_secret = 'senha2'
	conecta_ssh = paramiko.SSHClient()            # Cria uma sessão SSH
	conecta_ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
	arq_switches = open("switch.txt", "r")        # Leitura do arquivo texto com o IP dos switches
	for aline in arq_switches:                    # Lê todas as linhas do arquivo texto
		dados = aline.split(",")                    # Define a virgula como caracter separador dos campos no arquivo texto
		ip_switch = dados[0]                        # Cria variável com o IP do switch
		hostname = dados[1].rstrip()                # Cria variável com o nome do switch
		try:
			conecta_ssh.connect(ip_switch, username=login, password=pwd, look_for_keys=False, allow_agent=False)  # Faz a conexão com o switch
			comando = conecta_ssh.invoke_shell()      # Cria uma instância para execução dos comandos
		except AuthenticationException:             # Se ocorrer uma falha de autenticação irá adicionar uma mensagem na lista de erros
			erro = ' --> Falha de autenticação.'
			erro = ip_switch + erro
			lista_erros.append(erro)
			continue
		except:  # Qualquer outro erro que ocorre no acesso ao switch também irá adicionar uma mensagem na lista de erros
			erro = ' --> Backup não realizado.'
			erro = ip_switch + erro
			lista_erros.append(erro)
			continue

		# Envio dos comandos para acessar o modo de configuração do switch
		comando.send('\n')
		comando.send('en\n')
		comando.send(pwd_secret + '\n')
		comando.send(
			'show run | redirect tftp://X.X.X.X/' + hostname + '--' + ip_switch + '.cfg\n')  # Envia uma cópia do running config para o servidor de TFTP

		# Faz uma nova conexão no switch por outro método para capturar o hostname (única forma que consegui)
		get_name = {'device_type': 'cisco_ios', 'ip': ip_switch, 'username': login, 'password': pwd,
					'secret': pwd_secret}
		net_connect = ConnectHandler(**get_name)
		net_connect.enable()
		name = net_connect.send_command("show run | in hostname")  # Envia o comando para verificar o hostname do switch
		name = name.split()
		get_hostname = name[1]        # Associa o hostname do switch a uma variável
		if get_hostname != hostname:  # Compara o hostname do switch com o nome que está na planilha
			erro = ' --> O hostname do switch não é o mesmo nome que está no arquivo texto.'
			erro = ip_switch + erro
			lista_erros.append(erro)    # Adiciona a informação de hostname incorreto na lista de erros

	arq_switches.close()
lista_erros = []     #Cria um array vazio para inserir o IP dos switches quando ocorrer falha
conectaSwitch()
```

Usando NetMiko:
```
from netmiko import ConnectHandler

cisco_881 = {
    'device_type': 'cisco_ios',
    'host':   '10.10.10.10',
    'username': 'test',
    'password': 'password',
    'port' : 8022,          # optional, defaults to 22
    'secret': 'secret',     # optional, defaults to ''
}

net_connect = ConnectHandler(**cisco_881)

output = net_connect.send_command('show ip int brief')
print(output)
```